Acceleromter Stuff
====
Initial investigation

In [2]:
""" Find the right file """
import pathlib

device_id = "6012582"
recording_id = "0000291121"
participant_id = "20029"

# Check IDs are the right length
assert len(str(device_id)) == 7
assert len(str(recording_id)) == 10

# TODO Check that the participant agreed to take part in the study
# Do this by opening "Z:\SEACO data\SEACO-CH20 qnaire data\SEACO_CH20_17082022_de_id.csv"
# as a dataframe, finding the row with participant_id = df["residents_id"] and checking
# that the value of "respondent_status" here == 1
# I've checked the one above by hand and it's fine
...

filename = f"{device_id}_{recording_id}-{participant_id}.cwa"
# Windows path to RDSF mount on Z:
# Let's hope it's in the Week 1 folder
filepath = pathlib.Path(fr"Z:\SEACO data\Downloads\AX6 Activity Monitor\Week 1\{filename}")

# TODO add some code to look for the right file in all the "Week X" folders

assert filepath.exists()